In [1]:
pip install kaggle --upgrade

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)


In [2]:
#colab에서 g란 이름으로 구글드라이브의 root를 MOUNT
from google.colab import drive
drive.mount('/content/g')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/g


In [3]:
!mkdir -p ~/.kaggle
!cp "/content/g/My Drive/kaggle/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 21.8MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 28.2MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 6.29MB/s]


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

In [6]:
from sklearn import model_selection
from sklearn import metrics
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import linear_model

In [7]:
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

In [8]:
del df_train['PassengerId']
del df_train['Name']
del df_train['Ticket']
del df_train['Cabin']
df_test['PassengerId']
del df_test['Name']
del df_test['Ticket']
del df_test['Cabin']

In [9]:
df_train.dropna(thresh=int(len(df_train)*0.5), axis=1)
df_train['Embarked']=df_train['Embarked'].fillna('S')
df_train['Age']=df_train['Age'].fillna(df_train['Age'].mean())

In [10]:
df_test.dropna(thresh=int(len(df_test)*0.5), axis=1)
df_test['Fare']=df_test['Fare'].fillna(df_test['Fare'].mean())
df_test['Age']=df_test['Age'].fillna(df_test['Age'].mean())

In [11]:
df_train['Sex']=df_train['Sex'].astype('category')
df_train['Pclass']=df_train['Pclass'].astype('category')
df_train['Embarked']=df_train['Embarked'].astype('category')
df_train = pd.get_dummies(df_train)

df_test['Sex']=df_test['Sex'].astype('category')
df_test['Pclass']=df_test['Pclass'].astype('category')
df_test['Embarked']=df_test['Embarked'].astype('category')
df_test = pd.get_dummies(df_test)

In [12]:
x_data = df_train.iloc[:,1:]
y_data = df_train.iloc[:,0]

x_data = x_data.values
y_data = y_data.values
print(x_data)
print(y_data)

[[22.          1.          0.         ...  0.          0.
   1.        ]
 [38.          1.          0.         ...  1.          0.
   0.        ]
 [26.          0.          0.         ...  0.          0.
   1.        ]
 ...
 [29.69911765  1.          2.         ...  0.          0.
   1.        ]
 [26.          0.          0.         ...  1.          0.
   0.        ]
 [32.          0.          0.         ...  0.          1.
   0.        ]]
[0 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1
 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0
 1 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1
 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0
 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 

In [13]:
x2_data = df_test.iloc[:,1:]

x2_data = x2_data.values
print(x2_data)

[[34.5         0.          0.         ...  0.          1.
   0.        ]
 [47.          1.          0.         ...  0.          0.
   1.        ]
 [62.          0.          0.         ...  0.          1.
   0.        ]
 ...
 [38.5         0.          0.         ...  0.          0.
   1.        ]
 [30.27259036  0.          0.         ...  0.          0.
   1.        ]
 [30.27259036  1.          1.         ...  1.          0.
   0.        ]]


RandomForestClassifier  
n_estimators=100, criterion="gini", max_depth=None, min_samples_split=2,
                 min_samples_leaf=1,
                 min_weight_fraction_leaf=0.,
                 max_features="auto",
                 max_leaf_nodes=None,
                 min_impurity_decrease=0.,
                 min_impurity_split=None,
                 bootstrap=True,
                 oob_score=False,
                 n_jobs=None,
                 random_state=None,
                 verbose=0,
                 warm_start=False,
                 class_weight=None,
                 ccp_alpha=0.0,
                 max_samples=None

n_estimators	- 결정트리의 갯수를 지정
- Default = 10
- 무작정 트리 갯수를 늘리면 성능 좋아지는 것 대비 시간이 걸릴 수 있음  

min_samples_split	- 노드를 분할하기 위한 최소한의 샘플 데이터수
→ 과적합을 제어하는데 사용
- Default = 2 → 작게 설정할 수록 분할 노드가 많아져 과적합 가능성 증가  

min_samples_leaf	- 리프노드가 되기 위해 필요한 최소한의 샘플 데이터수
- min_samples_split과 함께 과적합 제어 용도
- 불균형 데이터의 경우 특정 클래스의 데이터가 극도로 작을 수 있으므로 작게 설정 필요  

max_features	- 최적의 분할을 위해 고려할 최대 feature 개수
- Default = 'auto' (결정트리에서는 default가 none이었음)
- int형으로 지정 →피처 갯수 / float형으로 지정 →비중
- sqrt 또는 auto : 전체 피처 중 √(피처개수) 만큼 선정
- log : 전체 피처 중 log2(전체 피처 개수) 만큼 선정  

max_depth	- 트리의 최대 깊이
- default = None
→ 완벽하게 클래스 값이 결정될 때 까지 분할
또는 데이터 개수가 min_samples_split보다 작아질 때까지 분할
- 깊이가 깊어지면 과적합될 수 있으므로 적절히 제어 필요  

max_leaf_nodes	리프노드의 최대 개수

In [14]:
from sklearn.model_selection import GridSearchCV

param_grid = [
  {'n_estimators' : [10, 100], 'max_depth' : [6, 8, 10, 12], 'min_samples_leaf' : [8, 12, 18], 'min_samples_split' : [8, 16, 20]}
]

forest_reg = RandomForestClassifier()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(x_data, y_data)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [15]:
grid_search.best_params_

{'max_depth': 10,
 'min_samples_leaf': 12,
 'min_samples_split': 20,
 'n_estimators': 10}

In [16]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
  print(np.sqrt(-mean_score), params)

0.4456599236638885 {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 10}
0.43292843411219073 {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 100}
0.4355088322918442 {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 16, 'n_estimators': 10}
0.4316215431223883 {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 16, 'n_estimators': 100}
0.4277282761870589 {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 20, 'n_estimators': 10}
0.43164335706337825 {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 20, 'n_estimators': 100}
0.44063854237535505 {'max_depth': 6, 'min_samples_leaf': 12, 'min_samples_split': 8, 'n_estimators': 10}
0.4393688730337225 {'max_depth': 6, 'min_samples_leaf': 12, 'min_samples_split': 8, 'n_estimators': 100}
0.45441915376557374 {'max_depth': 6, 'min_samples_leaf': 12, 'min_samples_split': 16, 'n_estimators': 10}
0.4469398105733168 {'max_depth': 6, 'min_samples_leaf': 1

In [17]:
forest = RandomForestClassifier(n_estimators=10, max_depth = 8, min_samples_leaf = 8, min_samples_split = 20)
forest.fit(x_data, y_data)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=8, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8, min_samples_split=20,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [18]:
y_predict = forest.predict(x2_data)

In [19]:
y_predict

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

각 특성의 상대적인 중요도

In [23]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([0.07857141, 0.02961951, 0.02156484, 0.16137637, 0.04620123,
       0.02065801, 0.06226376, 0.4483512 , 0.09767221, 0.0167208 ,
       0.00772901, 0.00927165])

In [40]:
sorted(zip(feature_importances, list(df_test)[1:]), reverse=True)

[(0.4483511981137679, 'Sex_female'),
 (0.16137637415613038, 'Fare'),
 (0.097672214260589, 'Sex_male'),
 (0.07857140648717023, 'Age'),
 (0.06226376229631971, 'Pclass_3'),
 (0.04620122791935432, 'Pclass_1'),
 (0.02961950579722538, 'SibSp'),
 (0.021564842157239698, 'Parch'),
 (0.020658011755971678, 'Pclass_2'),
 (0.016720796706554478, 'Embarked_C'),
 (0.00927164567721492, 'Embarked_S'),
 (0.00772901467246233, 'Embarked_Q')]

In [20]:
df_test

,PassengerId,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,34.50000,0,0,7.8292,0,0,1,0,1,0,1,0
1,893,47.00000,1,0,7.0000,0,0,1,1,0,0,0,1
2,894,62.00000,0,0,9.6875,0,1,0,0,1,0,1,0
3,895,27.00000,0,0,8.6625,0,0,1,0,1,0,0,1
4,896,22.00000,1,1,12.2875,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,30.27259,0,0,8.0500,0,0,1,0,1,0,0,1
414,1306,39.00000,0,0,108.9000,1,0,0,1,0,1,0,0
415,1307,38.50000,0,0,7.2500,0,0,1,0,1,0,0,1
416,1308,30.27259,0,0,8.0500,0,0,1,0,1,0,0,1


In [21]:
submission=pd.DataFrame({"PassengerId":df_test["PassengerId"], "Survived":y_predict})

In [22]:
submission.to_csv('submission.csv', index=False)